In [ ]:
!pip install cython
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!git clone https://github.com/pytorch/vision.git
!cd vision
!git checkout v0.3.0
!ls -l

In [ ]:
!cp ./vision/references/detection/utils.py .
!cp ./vision/references/detection/transforms.py .
!cp ./vision/references/detection/coco_eval.py .
!cp ./vision/references/detection/engine.py .
!cp ./vision/references/detection/coco_utils.py .

In [ ]:
!ls ./vision/references/detection/coco_utils.py

In [ ]:
!ls ./engine.py

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [ ]:
from __future__ import print_function
from collections import defaultdict, deque
import datetime
import pickle
import time
import torch.distributed as dist
import errno
from shapely.geometry import Polygon
import collections
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm
from torchvision import transforms
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from matplotlib.patches import Rectangle
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import io, transform
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
import psutil

import warnings
warnings.filterwarnings('ignore')

In [ ]:
input_path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'

In [ ]:
input_file = os.listdir(input_path)
input_files = list(dict.fromkeys(input_file))
print(len(input_files))

In [ ]:
df1 = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')

In [ ]:
#classes = list(df1['class_name'].values)
#classes = list(dict.fromkeys(classes))

In [ ]:
df = df1[df1['class_name']!= 'No finding']

In [ ]:
def plot_maskimage(mask):
    color = ['r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m']
    #plt.imshow(img,cmap='gray')
    #for ii in range(size):
    #    plt.gca().add_patch(Rectangle((xmin[ii], ymin[ii]), (xmax[ii]-xmin[ii]), (ymax[ii]-ymin[ii]),linewidth=1,edgecolor=color[i],facecolor='none'))
    #plt.show()
    for mas in mask:
        plt.imshow(mas,cmap='gray')
        plt.show()

In [ ]:
num_classes = 15

In [ ]:
def masks_generation(img,dff,res):
    m = img.shape[0]
    n = img.shape[1]
    #all_classids = list(np.arange(0,15))
    class_ids = list(dff['class_id'].values)
    mask = np.zeros((num_classes,m,n))
    print(class_ids)
    #for s in all_classids:
    #print(s)
    res = res
    for dd in dff.values:
        #print('dd',dd)
        s = dd[2]+1
        print('s',s)
        xmin = dd[4]/res
        print(dd[4],xmin)
        xmax = dd[5]/res
        ymin = dd[6]/res
        ymax = dd[7]/res
        ax = int(xmin)
        ax1 =int(xmax)
        ay = int(ymin)
        ay1 = int(ymax)
        mask[s,ay:ay1,ax:ax1] = 1
        print(mask.shape)
    #print(xmin,xmax,ymin,ymax)
        print(ax,ax1,ay,ay1)
    plot_maskimage(img,mask,num_classes)
    return mask

In [ ]:
class traindataset(torch.utils.data.Dataset):

    def __init__(self, df, transform):
        super().__init__()
        
        self.input_files = list(set(df['image_id'].values))
        self.df = df
        self.transform = transform
    
    def __len__(self) :
        return len(self.input_files)

    def __getitem__(self, idx):
        
        input_files = self.input_files
        #print(idx)
        img_id = input_files[idx]
        df = self.df
        
        fdf = df[df['image_id'] == img_id]
        m,n = 1024,1024
        dff = fdf.sort_values(by=['x_min'])
        class_id = [m+1 for m in list(dff['class_id'].values)]
        N = len(class_id)
        
        image_path  = '../input/vinbigdata-chest-xray-abnormalities-detection/train/'+img_id+'.dicom'
        img_pxl = pydicom.read_file(image_path).pixel_array
        res = (img_pxl.shape[0]*img_pxl.shape[1])/(m*n) 
        img = resize(img_pxl,(1024,1024),anti_aliasing=True)
        img_np = img.astype(np.float32())
        img_tr = torch.from_numpy(img_np).unsqueeze(0)

        xmin_res = [x/res for x in list(dff['x_min'].values)]
        xmax_res = [xm/res for xm in list(dff['x_max'].values)]
        ymax_res = [y/res for y in list(dff['y_max'].values)]
        ymin_res = [ym/res for ym in list(dff['y_min'].values)]
        b = 0
        area = []
        bbox = []
        mask = np.zeros((N,m,n))
        for b in range(N):
            x1,x2,y1,y2 = int(xmin_res[b]),int(xmax_res[b]),int(ymin_res[b]),int(ymax_res[b])
            ax1,ax2,ay1,ay2 = xmin_res[b],xmax_res[b],ymin_res[b],ymax_res[b]
            mask[b,y1:y2,x1:x2] = 1
            box = [ax1,ay1,ax2,ay2]
            a = (ax2-ax1)*(ay2-ay1)
            area.append(a)
            bbox.append(box)

        boxes = torch.as_tensor(bbox, dtype=torch.int64)
        labels = torch.as_tensor(class_id, dtype=torch.int64)
        masks = torch.as_tensor(mask, dtype=torch.uint8)
        image_id = torch.tensor([idx])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        
        return img_tr, target
    
traindata = traindataset(df,transform = None)

In [ ]:
len(traindata)

In [ ]:
data_loader = torch.utils.data.DataLoader(traindata, batch_size=2, shuffle=True, num_workers=2,collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
data_frame = df[df['class_name'] == 'Pleural effusion']

In [ ]:
class validdataset(torch.utils.data.Dataset):

    def __init__(self, data_frame, transform):
        super().__init__()
        
        self.input_files = list(set(data_frame['image_id'].values))
        self.data_frame = data_frame
        self.transform = transform
    
    def __len__(self) :
        return len(self.input_files)

    def __getitem__(self, idx):
        
        input_files = self.input_files
        #print(idx)
        img_id = input_files[10]
        df = self.data_frame
        
        fdf = df[df['image_id'] == img_id]
        m,n = 1024,1024
        dff = fdf.sort_values(by=['x_min'])
        class_id = [m+1 for m in list(dff['class_id'].values)]
        N = len(class_id)
        
        image_path  = '../input/vinbigdata-chest-xray-abnormalities-detection/train/'+img_id+'.dicom'
        img_pxl = pydicom.read_file(image_path).pixel_array
        res = (img_pxl.shape[0]*img_pxl.shape[1])/(m*n) 
        img = resize(img_pxl,(1024,1024),anti_aliasing=True)
        img_np = img.astype(np.float32())
        img_tr = torch.from_numpy(img_np).unsqueeze(0)

        xmin_res = [x/res for x in list(dff['x_min'].values)]
        xmax_res = [xm/res for xm in list(dff['x_max'].values)]
        ymax_res = [y/res for y in list(dff['y_max'].values)]
        ymin_res = [ym/res for ym in list(dff['y_min'].values)]
        b = 0
        bbox = []
        for b in range(N):
            ax1,ax2,ay1,ay2 = xmin_res[b],xmax_res[b],ymin_res[b],ymax_res[b]
            box = [ax1,ay1,ax2,ay2]
            bbox.append(box)
        return img, img_tr, bbox
    
validdata = validdataset(data_frame,transform = None)
validdata_loader = torch.utils.data.DataLoader(validdata, batch_size=1, shuffle=True, num_workers=1)

In [ ]:
test_files = os.listdir('../input/vinbigdata-chest-xray-abnormalities-detection/test/')
class testdataset(torch.utils.data.Dataset):

    def __init__(self, test_files, transform):
        super().__init__()

        self.test_files = test_files
        self.transform = transform
        
    def __len__(self) -> int:
        return len(test_files)

    def __getitem__(self, index: int):
        img_id = self.test_files[index]
        image_path  = '../input/vinbigdata-chest-xray-abnormalities-detection/test/'+img_id
        dicom_data = pydicom.read_file(image_path)
        img_pixel = dicom_data.pixel_array 
        image = resize(img_pixel,(1024,1024),anti_aliasing=True)
        img_numpy = image.astype(np.float32())
        img_tensor = torch.from_numpy(img_numpy).unsqueeze(0)

        return img_tensor.unsqueeze(0)
testdata = testdataset(test_files,transform = None)
test_loader = torch.utils.data.DataLoader(testdata, batch_size=1, shuffle=True, num_workers=1,collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
model = get_model_instance_segmentation(num_classes)

In [ ]:
device = torch.device('cuda:0')
model.to(device)

In [ ]:
model.load_state_dict(torch.load('../input/mask-rcnn-fop/mask_rcnn_sd1 (1).pth'))
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)
num_epochs = 5
loss_values = []
for epoch in range(num_epochs):
    metric_logger = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=2197)
    loss_values.append(metric_logger.meters['loss'].median)
    torch.save(model.state_dict(),'./mask_rcnn_sd'+str(epoch)+'.pth')
    torch.save(model,'./mask_rcnn'+str(epoch)+'.pth')
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    #evaluate(model, test, device=device)

In [ ]:
loss_valueso = [0.8922919034957886, 1.0178143978118896, 1.0720345973968506, 0.9409962892532349, 1.1191381216049194]

In [ ]:
print(metric_logger)
print(metric_logger.meters['loss'].median)

In [ ]:
model.load_state_dict(torch.load('./mask_rcnn_sd4.pth'))

In [ ]:
def plot_image(img,bboxes):
    img_resize = img
    bbox = bboxes
    color = ['r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m']
    plt.imshow(img_resize,cmap='gray')
    for m,p in enumerate(bbox):
        plt.gca().add_patch(Rectangle((p[0], p[1]), (p[2]-p[0]), (p[3]-p[1]),linewidth=1,edgecolor=color[m],facecolor='none'))
    plt.show()

In [ ]:
model.eval()

In [ ]:
for pred in validdata:
    img_o = pred[0]
    image_tst = pred[1]
    output = model(pred[1].unsqueeze(0).to(device))
    pred_boxes = output[0]['boxes'].cpu().detach().numpy()
    pred_scores = list(output[0]['scores'].cpu().detach().numpy())
    #print(pred_scores)
    predictions = [pred_scores.index(p) for p in pred_scores if p > 0.7]
    print(predictions)
    bboxes = []
    predscores = []
    for val in predictions:
        bboxes.append(pred_boxes[val])
        predscores.append(pred_scores[val])
    o_box = pred[2]
    plot_image(img_o,o_box)
    plot_image(img_o,bboxes)
    break

In [ ]:
dfff = df.iloc[15]
img_tst = pydicom.read_file('../input/vinbigdata-chest-xray-abnormalities-detection/train/'+dfff['image_id']+'.dicom').pixel_array

In [ ]:
image_tst = resize(img_tst,(1024,1024),anti_aliasing=True)
img_numpy_tst = image_tst.astype(np.float32())
img_tensor_tst = torch.from_numpy(img_numpy_tst).unsqueeze(0)

In [ ]:
model.eval()
output = model(img_tensor_tst.unsqueeze(0).to(device))

In [ ]:
output

In [ ]:
def plot_image(img,bboxes):
    img_resize = img
    bbox = bboxes
    color = ['r','b','g','y','w','c','m','w','c','m']
    plt.imshow(img_resize,cmap='gray')
    for m,p in enumerate(bbox):
        plt.gca().add_patch(Rectangle((p[0], p[1]), (p[2]-p[0]), (p[3]-p[1]),linewidth=1,edgecolor=color[m],facecolor='none'))
    plt.show()

In [ ]:
def plot_o_image(img,dd):
    img_resize = img
    dd = dd
    xmin_o = list(dd['x_min'].values)
    ymin_o = list(dd['y_min'].values)
    xmax_o = list(dd['x_max'].values)
    ymax_o = list(dd['y_max'].values)
    bbox = bboxes
    color = ['r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m','r','b','g','y','w','c','m','w','c','m']
    plt.imshow(img_resize,cmap='gray')
    print(xmin_o)
    for i in range(len(xmin_o)):
        plt.gca().add_patch(Rectangle((xmin_o[i], ymin_o[i]), (xmax_o[i]-xmin_o[i]), (ymax_o[i]-ymin_o[i]),linewidth=1,edgecolor=color[i],facecolor='none'))
    plt.show()

In [ ]:
output

In [ ]:
pred_boxes = output[0]['boxes'].cpu().detach().numpy()
pred_scores = list(output[0]['scores'].cpu().detach().numpy())
print(pred_scores)
predictions = [pred_scores.index(p) for p in pred_scores if p > 0.4]
print(predictions)
bboxes = []
predscores = []
for val in predictions:
    bboxes.append(pred_boxes[val])
    predscores.append(pred_scores[val])

In [ ]:
dd = df[df['image_id'] == dfff['image_id']]
dd = dd.sort_values(by=['x_min'])
print(dd)

In [ ]:
print(bboxes,predscores)
plot_o_image(img_tst,dd)
plot_image(image_tst,bboxes)

In [ ]:
pred_boxes

In [ ]:
#!pip install pycocotools

In [ ]:
class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    # receiving Tensor from all ranks
    # we pad the tensor because torch all_gather does not support
    # gathering tensors of different shapes
    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(size=(max_size - local_size,), dtype=torch.uint8, device="cuda")
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        log_msg = self.delimiter.join([
            header,
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}',
            'time: {time}',
            'data: {data}',
            'max mem: {memory:.0f}'
        ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time),
                    memory=torch.cuda.max_memory_allocated() / MB))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def collate_fn(batch):
    return tuple(zip(*batch))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print('image_id',targets[0]['image_id'])
        #print('image--',images[0].shape)
        #print('targets--',targets[0]['boxes'].shape,targets[0]['labels'],targets[0]['masks'].shape)
        loss_dict = model(images, targets)
        #print('loss_dict-----',loss_dict)
        #print('epoch')
        losses = sum(loss for loss in loss_dict.values())/len(loss_dict.values())
        #print('losses-----',losses)
        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        #print('losses_reduced-----',losses_reduced)
        optimizer.zero_grad()
        #print('zero grad over')
        losses.backward()
        #print('losses backward over')
        optimizer.step()
        #print('optimizer step over')
        if lr_scheduler is not None:
            #print('lr_scheduler',lr_scheduler)
            lr_scheduler.step()
            #print('lr_scheduler')
        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        print('before del',psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
        del losses_reduced,loss_dict_reduced,images,targets
        print('after del',psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)

In [ ]:
device = torch.device('cuda:0')
#device = torch.device('cpu')

#model_ft = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
#in_features = model_ft.roi_heads.box_predictor.cls_score.in_features
#print('infeatures',in_features)
#model_ft.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
#in_features_mask = model_ft.roi_heads.mask_predictor.conv5_mask.in_channels
#hidden_layer = 256
#model_ft.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
model_ft = model
model_ft.to(device)

for param in model_ft.parameters():
    param.requires_grad = True

In [ ]:
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)

In [ ]:
next(model_ft.parameters()).is_cuda

In [ ]:
#model_ft.load_state_dict(torch.load('../input/maskrcnn-op-1/previous_model_sd (1).pth'))

In [ ]:
optimizer = optim.Adam(model_ft.parameters())
train_losses = []
epochs = 4
for epoch in range(epochs):
    model_ft.train()
    running_loss = 0
    size = 0
    print('epoch',epoch)
    for a,i in enumerate(data_loader):
        #print('iter number'+str(a)+' for epoch '+str(epoch))
        images = list(image.to(device) for image in i[0])
        targets = [{k: v.to(device) for k, v in t.items()} for t in i[1]]
        
        #model_ft.eval()
        #print('targets',targets[0]['boxes'])
        #output = model_ft(images)
        #print('outputs',output[0]['boxes'])
        
        model_ft.train()
        optimizer.zero_grad()
        loss_dict = model_ft(images,targets)
        losses = sum(loss for loss in loss_dict.values())/len(loss_dict.values())
        print('loss for batch_no '+str(a),losses)
        running_loss += losses.item()
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model_ft.parameters(), 1000000)
        optimizer.step()
        #print('targets',targets[0]['boxes'])
        del images,targets,loss_dict
    train_losses.append(float(running_loss)/len(data_loader))
    torch.save(model_ft.state_dict(),'./previous_model_sd.pth')
    torch.save(model_ft,'./'+str(epoch)+'mask_rcnn.pth')
    print(epoch,'epoch completed')
    print('train_looses',train_losses)
    #print('space',psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
torch.save(model_ft.state_dict(),'./final_mask_rcnn_sd.pth')
torch.save(model_ft,'./final_mask_rcnn.pth')

In [ ]:
#!cat /opt/conda/lib/python3.7/site-packages/torchvision/models/detection/roi_heads.py

In [ ]:
#!cat /opt/conda/lib/python3.7/site-packages/torchvision/models/detection/roi_heads.py

In [ ]:
#model_ft.eval()

In [ ]:
#for b,val in enumerate(data_loader):
#    image = list(img.to(device) for img in val[0])
#    targets = [{k: v.to(device) for k, v in t.items()} for t in val[1]]
#    outputs = model_ft(image)
#    outputs = [{k: v for k, v in t.items()} for t in outputs]
    #if len(outputs[0]['boxes']) >= 1:
#    print('target',targets[0]['boxes'])
#    print('output',outputs[0]['boxes'])

images = glob.glob('../input/vinbigdata-chest-xray-abnormalities-detection/train/*.dicom')

for i in images:
    j = pydicom.read_file(i).pixel_array
    mask = np.ones(j.shape)
    mask[50:100,150:250] = 0
    plt.imshow(j*mask)
    plt.show()
    print(j)
    break

print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)